In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_time_loss(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(100)
test_Y = user_function_time_loss(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(1020)
train_Y = user_function_time_loss(train_X)

[[16.         17.32419781 23.33514973 57.56156511  3.          3.14597424]
 [10.         12.26304583 11.61095924 65.88599641  3.          3.63207691]
 [ 6.         20.0319877   9.79580612 38.17820559  1.          2.95302089]
 [13.         20.96986397  8.74229956 50.10693532  1.          4.43718855]
 [13.          9.80556849 14.79632727 36.52599483  3.          1.7321506 ]
 [ 9.          8.16396631 22.6507855  44.7344982   1.          3.82989305]
 [ 9.         12.16172735 19.23982191 40.60155571  1.          2.92092508]
 [16.         13.48818192 22.04754469 45.32869228  2.          4.90844362]
 [11.         12.83608539 26.19564389 30.63727673  1.          3.50134549]
 [18.         24.89348029 12.02839137 34.35069442  1.          4.13446162]
 [ 4.         18.88181839 18.57684695 55.20389921  1.          4.86986006]
 [ 4.         10.33969149 25.45238377 31.7617212   3.          3.70844161]
 [14.         11.85692983 24.30524994 53.48709549  2.          4.37356066]
 [18.         14.37769663

 Retrying in 1 seconds

Evaluating input: 2 of 100

 Retrying in 1 seconds

Evaluating input: 3 of 100

 Retrying in 1 seconds

Evaluating input: 4 of 100

 Retrying in 1 seconds

Evaluating input: 5 of 100

 Retrying in 1 seconds

Evaluating input: 6 of 100

 Retrying in 1 seconds

Evaluating input: 7 of 100

 Retrying in 1 seconds

Evaluating input: 8 of 100

 Retrying in 1 seconds

Evaluating input: 9 of 100

 Retrying in 1 seconds

Evaluating input: 10 of 100

 Retrying in 1 seconds

Evaluating input: 11 of 100

 Retrying in 1 seconds

Evaluating input: 12 of 100

 Retrying in 1 seconds

Evaluating input: 13 of 100

 Retrying in 1 seconds

Evaluating input: 14 of 100

 Retrying in 1 seconds

Evaluating input: 15 of 100

 Retrying in 1 seconds

Evaluating input: 16 of 100

 Retrying in 1 seconds

Evaluating input: 17 of 100

 Retrying in 1 seconds

Evaluating input: 18 of 100

 Retrying in 1 seconds

Evaluating input: 19 of 100

 Retrying in 1 seconds

Evaluating input: 20 of 100

 


Evaluating input: 46 of 1020

 Retrying in 1 seconds

Evaluating input: 47 of 1020

 Retrying in 1 seconds

Evaluating input: 48 of 1020

 Retrying in 1 seconds

Evaluating input: 49 of 1020

 Retrying in 1 seconds

Evaluating input: 50 of 1020

 Retrying in 1 seconds

Evaluating input: 51 of 1020

 Retrying in 1 seconds

Evaluating input: 52 of 1020

 Retrying in 1 seconds

Evaluating input: 53 of 1020

 Retrying in 1 seconds

Evaluating input: 54 of 1020

 Retrying in 1 seconds

Evaluating input: 55 of 1020

 Retrying in 1 seconds

Evaluating input: 56 of 1020

 Retrying in 1 seconds

Evaluating input: 57 of 1020

 Retrying in 1 seconds

Evaluating input: 58 of 1020

 Retrying in 1 seconds

Evaluating input: 59 of 1020

 Retrying in 1 seconds

Evaluating input: 60 of 1020

 Retrying in 1 seconds

Evaluating input: 61 of 1020

 Retrying in 1 seconds

Evaluating input: 62 of 1020

 Retrying in 1 seconds

Evaluating input: 63 of 1020

 Retrying in 1 seconds

Evaluating input: 64 of 102


Evaluating input: 196 of 1020

 Retrying in 1 seconds

Evaluating input: 197 of 1020

 Retrying in 1 seconds

Evaluating input: 198 of 1020

 Retrying in 1 seconds

Evaluating input: 199 of 1020

 Retrying in 1 seconds

Evaluating input: 200 of 1020

 Retrying in 1 seconds

Evaluating input: 201 of 1020

 Retrying in 1 seconds

Evaluating input: 202 of 1020

 Retrying in 1 seconds

Evaluating input: 203 of 1020

 Retrying in 1 seconds

Evaluating input: 204 of 1020

 Retrying in 1 seconds

Evaluating input: 205 of 1020

 Retrying in 1 seconds

Evaluating input: 206 of 1020

 Retrying in 1 seconds

Evaluating input: 207 of 1020

 Retrying in 1 seconds

Evaluating input: 208 of 1020

 Retrying in 1 seconds

Evaluating input: 209 of 1020

 Retrying in 1 seconds

Evaluating input: 210 of 1020

 Retrying in 1 seconds

Evaluating input: 211 of 1020

 Retrying in 1 seconds

Evaluating input: 212 of 1020

 Retrying in 1 seconds

Evaluating input: 213 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 345 of 1020

 Retrying in 1 seconds

Evaluating input: 346 of 1020

 Retrying in 1 seconds

Evaluating input: 347 of 1020

 Retrying in 1 seconds

Evaluating input: 348 of 1020

 Retrying in 1 seconds

Evaluating input: 349 of 1020

 Retrying in 1 seconds

Evaluating input: 350 of 1020

 Retrying in 1 seconds

Evaluating input: 351 of 1020

 Retrying in 1 seconds

Evaluating input: 352 of 1020

 Retrying in 1 seconds

Evaluating input: 353 of 1020

 Retrying in 1 seconds

Evaluating input: 354 of 1020

 Retrying in 1 seconds

Evaluating input: 355 of 1020

 Retrying in 1 seconds

Evaluating input: 356 of 1020

 Retrying in 1 seconds

Evaluating input: 357 of 1020

 Retrying in 1 seconds

Evaluating input: 358 of 1020

 Retrying in 1 seconds

Evaluating input: 359 of 1020

 Retrying in 1 seconds

Evaluating input: 360 of 1020

 Retrying in 1 seconds

Evaluating input: 361 of 1020

 Retrying in 1 seconds

Evaluating input: 362 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 494 of 1020

 Retrying in 1 seconds

Evaluating input: 495 of 1020

 Retrying in 1 seconds

Evaluating input: 496 of 1020

 Retrying in 1 seconds

Evaluating input: 497 of 1020

 Retrying in 1 seconds

Evaluating input: 498 of 1020

 Retrying in 1 seconds

Evaluating input: 499 of 1020

 Retrying in 1 seconds

Evaluating input: 500 of 1020

 Retrying in 1 seconds

Evaluating input: 501 of 1020

 Retrying in 1 seconds

Evaluating input: 502 of 1020

 Retrying in 1 seconds

Evaluating input: 503 of 1020

 Retrying in 1 seconds

Evaluating input: 504 of 1020

 Retrying in 1 seconds

Evaluating input: 505 of 1020

 Retrying in 1 seconds

Evaluating input: 506 of 1020

 Retrying in 1 seconds

Evaluating input: 507 of 1020

 Retrying in 1 seconds

Evaluating input: 508 of 1020

 Retrying in 1 seconds

Evaluating input: 509 of 1020

 Retrying in 1 seconds

Evaluating input: 510 of 1020

 Retrying in 1 seconds

Evaluating input: 511 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 643 of 1020

 Retrying in 1 seconds

Evaluating input: 644 of 1020

 Retrying in 1 seconds

Evaluating input: 645 of 1020

 Retrying in 1 seconds

Evaluating input: 646 of 1020

 Retrying in 1 seconds

Evaluating input: 647 of 1020

 Retrying in 1 seconds

Evaluating input: 648 of 1020

 Retrying in 1 seconds

Evaluating input: 649 of 1020

 Retrying in 1 seconds

Evaluating input: 650 of 1020

 Retrying in 1 seconds

Evaluating input: 651 of 1020

 Retrying in 1 seconds

Evaluating input: 652 of 1020

 Retrying in 1 seconds

Evaluating input: 653 of 1020

 Retrying in 1 seconds

Evaluating input: 654 of 1020

 Retrying in 1 seconds

Evaluating input: 655 of 1020

 Retrying in 1 seconds

Evaluating input: 656 of 1020

 Retrying in 1 seconds

Evaluating input: 657 of 1020

 Retrying in 1 seconds

Evaluating input: 658 of 1020

 Retrying in 1 seconds

Evaluating input: 659 of 1020

 Retrying in 1 seconds

Evaluating input: 660 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 792 of 1020

 Retrying in 1 seconds

Evaluating input: 793 of 1020

 Retrying in 1 seconds

Evaluating input: 794 of 1020

 Retrying in 1 seconds

Evaluating input: 795 of 1020

 Retrying in 1 seconds

Evaluating input: 796 of 1020

 Retrying in 1 seconds

Evaluating input: 797 of 1020

 Retrying in 1 seconds

Evaluating input: 798 of 1020

 Retrying in 1 seconds

Evaluating input: 799 of 1020

 Retrying in 1 seconds

Evaluating input: 800 of 1020

 Retrying in 1 seconds

Evaluating input: 801 of 1020

 Retrying in 1 seconds

Evaluating input: 802 of 1020

 Retrying in 1 seconds

Evaluating input: 803 of 1020

 Retrying in 1 seconds

Evaluating input: 804 of 1020

 Retrying in 1 seconds

Evaluating input: 805 of 1020

 Retrying in 1 seconds

Evaluating input: 806 of 1020

 Retrying in 1 seconds

Evaluating input: 807 of 1020

 Retrying in 1 seconds

Evaluating input: 808 of 1020

 Retrying in 1 seconds

Evaluating input: 809 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 941 of 1020

 Retrying in 1 seconds

Evaluating input: 942 of 1020

 Retrying in 1 seconds

Evaluating input: 943 of 1020

 Retrying in 1 seconds

Evaluating input: 944 of 1020

 Retrying in 1 seconds

Evaluating input: 945 of 1020

 Retrying in 1 seconds

Evaluating input: 946 of 1020

 Retrying in 1 seconds

Evaluating input: 947 of 1020

 Retrying in 1 seconds

Evaluating input: 948 of 1020

 Retrying in 1 seconds

Evaluating input: 949 of 1020

 Retrying in 1 seconds

Evaluating input: 950 of 1020

 Retrying in 1 seconds

Evaluating input: 951 of 1020

 Retrying in 1 seconds

Evaluating input: 952 of 1020

 Retrying in 1 seconds

Evaluating input: 953 of 1020

 Retrying in 1 seconds

Evaluating input: 954 of 1020

 Retrying in 1 seconds

Evaluating input: 955 of 1020

 Retrying in 1 seconds

Evaluating input: 956 of 1020

 Retrying in 1 seconds

Evaluating input: 957 of 1020

 Retrying in 1 seconds

Evaluating input: 958 of 1020

 Retrying in 1 seconds

Evaluatin

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 4.758979433709968
Optimization restart 2/5, f = 4.758979446321362
Optimization restart 3/5, f = 4.75897944632251
Optimization restart 4/5, f = 4.75897944632219
Optimization restart 5/5, f = 4.758979446322188


GP_regression.,value,constraints,priors
rbf.variance,0.047116765037615325,+ve,
rbf.lengthscale,1.0000001146683266,+ve,
Gaussian_noise.variance,0.047116764656467615,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 1000)

Optimization restart 1/5, f = 4.758979433709968
Optimization restart 2/5, f = 4.758979446322485
Optimization restart 3/5, f = 4.758977135501539
Optimization restart 4/5, f = 4.758979445436836
Optimization restart 5/5, f = 4.758979446322192
[[18.         20.28107878 15.99433057 59.89463116  2.          3.70337945]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 4.8862999840681915
Optimization restart 2/5, f = 4.886302316315636
Optimization restart 3/5, f = 4.88629843285022
Optimization restart 4/5, f = -23.245361894460014
Optimization restart 5/5, f = 4.886302316314499
[[ 3.  25.  27.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -23.871731000814442
Optimization restart 2/5, f = 6.397174718195586
Optimization restart 3/5, f = 6.397178828885183
Optimization restart 4/5, f = -23.871730906231

 Retrying in 1 seconds
Optimization restart 1/5, f = -38.828795080228225
Optimization restart 2/5, f = 9.641464131740491
Optimization restart 3/5, f = 9.641464131740484
Optimization restart 4/5, f = 9.641464089682017
Optimization restart 5/5, f = 9.641464131740491
[[ 3.         25.         27.         45.78812222  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -38.61036442161951
Optimization restart 2/5, f = 10.837193949951356
Optimization restart 3/5, f = -38.61036442141088
Optimization restart 4/5, f = 10.83719395384739
Optimization restart 5/5, f = -38.61036442160338
[[ 3.          8.         27.         59.40132374  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -40.99335116988882
Optimization restart 2/5, f = 10.762898083763417
Optimization restart 3/5,

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


 Retrying in 1 seconds
Optimization restart 1/5, f = -55.795920986163694
Optimization restart 2/5, f = 14.556497628555388
Optimization restart 3/5, f = 14.556495820329427
Optimization restart 4/5, f = -55.79592098575162
Optimization restart 5/5, f = 14.55649762855542
[[20.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -57.20215797856002
Optimization restart 2/5, f = -57.202157978127374
Optimization restart 3/5, f = 14.342602745044147
Optimization restart 4/5, f = -57.202157978458835
Optimization restart 5/5, f = -45.83985161089933
[[ 3.   8.   5.  30.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -58.61480242465518
Optimization restart 2/5, f = -58.61480242471145
Optimization restart 3/5, f = 14.159792100044559
Optimization restart 4/5, f = -58.61480242472534
Optim

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


...

Optimization restart 2/5, f = -1547.4533300497055
Optimization restart 3/5, f = -2051.5167198356567
Optimization restart 4/5, f = -2051.5167198358013
Optimization restart 5/5, f = -2051.5167198370345
[[15.         24.93132142 10.25289677 30.17225357  3.          4.80390934]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2052.020288278345
Optimization restart 2/5, f = -2052.021070779261
Optimization restart 3/5, f = -2052.0210707711344
Optimization restart 4/5, f = -2052.0210707803226
Optimization restart 5/5, f = -2052.021070340765
[[ 3.         24.71239527 26.95538523 56.877339    1.          1.66894837]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2054.9868115794384
Optimization restart 2/5, f = -2054.9868620071848
Optimization restart 3/5, f = -2054.9868620087286
Optimization restart 4/5, f = -1539

Optimization restart 2/5, f = -2093.294584314336
Optimization restart 3/5, f = -2093.2945843152893
Optimization restart 4/5, f = -2093.2945843150246
Optimization restart 5/5, f = -2093.2945843140706
[[ 3.         24.70382831 10.54877724 69.33871582  3.          1.97213868]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2096.176519822811
Optimization restart 2/5, f = -2096.1765409222266
Optimization restart 3/5, f = -2096.1765407042026
Optimization restart 4/5, f = -2096.176540922632
Optimization restart 5/5, f = -2096.1765409233417
[[ 3.         24.87051894 26.85889766 69.27330183  3.          2.24472001]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2098.7624840175436
Optimization restart 2/5, f = -1547.8163725732325
Optimization restart 3/5, f = -2098.7630932555153
Optimization restart 4/5, f = -2098

Optimization restart 2/5, f = -2141.5950222706324
Optimization restart 3/5, f = -2141.5950222721162
Optimization restart 4/5, f = -2141.595022271064
Optimization restart 5/5, f = -2141.595022270949
[[ 7.          8.13203894  9.07688885 63.27910682  3.          1.50761896]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2143.146048807607
Optimization restart 2/5, f = -2143.1471299720934
Optimization restart 3/5, f = -1591.9175323573415
Optimization restart 4/5, f = -2143.147129967099
Optimization restart 5/5, f = -2143.147129906265
[[14.         17.14058014 26.93185426 69.55723746  3.          4.97543607]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2146.1571098150957
Optimization restart 2/5, f = -2146.157125153762
Optimization restart 3/5, f = -2146.1571254773785
Optimization restart 4/5, f = -2146.15

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = -2181.693043096989
Optimization restart 3/5, f = -2181.693043091374
Optimization restart 4/5, f = -2181.693043088514
Optimization restart 5/5, f = -2181.6930430966445
[[ 3.          8.15930272 26.81353965 58.20061097  1.          3.32469671]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2184.711421919166
Optimization restart 2/5, f = -2184.711438875507
Optimization restart 3/5, f = -2184.7114388795267
Optimization restart 4/5, f = -2184.7114388795935
Optimization restart 5/5, f = -2184.711438879169
[[ 3.         18.45657619 26.88630573 30.5270256   1.          4.72507995]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2187.299408084834
Optimization restart 2/5, f = -2187.299795233758
Optimization restart 3/5, f = -1628.0424900705625
Optimization restart 4/5, f = -2187.2997

### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 4.758979433709968
Optimization restart 2/5, f = 4.7589794463302475
Optimization restart 3/5, f = 4.758979446321982
Optimization restart 4/5, f = 4.75897943239633
Optimization restart 5/5, f = 4.758979446315333


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [17]:
ed_loop.run_loop(user_function_time_loss, 1000)

Optimization restart 1/5, f = 4.75897943239633
Optimization restart 2/5, f = 4.75897944607345
Optimization restart 3/5, f = 4.758979446321943
Optimization restart 4/5, f = 4.75897944632219
Optimization restart 5/5, f = 4.758979446322499
[[16.         11.31205602  8.72009367 63.69084571  1.          3.0351125 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 4.575571815357668
Optimization restart 2/5, f = 4.575571829893326
Optimization restart 3/5, f = 4.57557182990279
Optimization restart 4/5, f = 4.575571829901879
Optimization restart 5/5, f = -23.624608945705162
[[11.         17.37063159 11.11712423 67.90083326  3.          3.5076603 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -24.43680136448745
Optimization restart 2/5, f = 4.461054162770695
Optimization restart 3/5, f = 4.461064899387136
Optimizat

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[13.         20.30483194  6.70465916 55.49037185  2.          2.0334001 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -32.039468791831126
Optimization restart 2/5, f = 4.149619485156915
Optimization restart 3/5, f = 4.149619485159036
Optimization restart 4/5, f = -30.132317131802537
Optimization restart 5/5, f = 4.149619485156968


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[ 7.          8.71102073 12.69377372 54.16459301  3.          4.2330655 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -34.15292486003899
Optimization restart 2/5, f = -34.1529276870699
Optimization restart 3/5, f = -34.152927698970245
Optimization restart 4/5, f = -34.152927892607295
Optimization restart 5/5, f = 4.0282647336364
[[20.         10.07626431 14.47071231 39.50921445  1.          3.80395263]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -36.3292674822654
Optimization restart 2/5, f = 3.9071991292301487
Optimization restart 3/5, f = 3.9071991292301593
Optimization restart 4/5, f = 3.907199129230161
Optimization restart 5/5, f = 3.9071991292320014
[[16.         16.71548187 15.17138712 66.10849704  1.          2.48057149]]

User function time loss called with 1 inputs to simulate

Evaluating 

...

Optimization restart 3/5, f = -2064.5013911556234
Optimization restart 4/5, f = -2064.5013912620807
Optimization restart 5/5, f = -2064.5013915048876
[[17.         22.81424159 17.82237815 62.59175319  1.          1.87341764]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2066.8055539352317
Optimization restart 2/5, f = -2066.8055686696757
Optimization restart 3/5, f = -2066.8055686714697
Optimization restart 4/5, f = -2066.805568671564
Optimization restart 5/5, f = -2066.8055686642087
[[10.         17.30837899  5.21120328 66.83241038  3.          1.50176232]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2069.6634387200543
Optimization restart 2/5, f = -2069.663447585247
Optimization restart 3/5, f = -2069.663448561778
Optimization restart 4/5, f = -2069.663447100179
Optimization restart 5/5, f = -2069.

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = -2091.181894411024
Optimization restart 3/5, f = -2091.1818944110246
Optimization restart 4/5, f = -2091.181894246069
Optimization restart 5/5, f = -2091.1818944111005
[[13.         16.72956253 23.27592207 48.58277235  3.          2.43569325]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2093.82203175453
Optimization restart 2/5, f = -2093.8220317468717
Optimization restart 3/5, f = -2093.8220317597375
Optimization restart 4/5, f = -2093.82203171737
Optimization restart 5/5, f = -2093.822031600669
[[20.          9.44476597 26.80095885 34.20301776  3.          2.5931334 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2096.5351687805005
Optimization restart 2/5, f = -2096.5351779869943
Optimization restart 3/5, f = -2096.53517798037
Optimization restart 4/5, f = -2096.53517

Optimization restart 2/5, f = -2140.9618150076485
Optimization restart 3/5, f = -2140.9618150083984
Optimization restart 4/5, f = -2140.9618150078054
Optimization restart 5/5, f = -2140.9618149998164
[[11.         18.95288734 10.9269908  49.51726419  3.          3.3286485 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2143.869671376621
Optimization restart 2/5, f = -2143.8696715076485
Optimization restart 3/5, f = -2143.869671564109
Optimization restart 4/5, f = -2143.8696715245655
Optimization restart 5/5, f = -2143.869671559477
[[13.         17.63998243 16.53448041 52.24303802  1.          4.24579364]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2146.789009667369
Optimization restart 2/5, f = -2146.7890244863656
Optimization restart 3/5, f = -2146.7890245648973
Optimization restart 4/5, f = -2146.

Optimization restart 2/5, f = -2191.8963690139212
Optimization restart 3/5, f = -2191.896369098762
Optimization restart 4/5, f = -2191.8963690986966
Optimization restart 5/5, f = -2191.8963690973756
[[ 7.         15.42389302 23.74402096 57.51054118  1.          3.88423572]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2194.6194686878075
Optimization restart 2/5, f = -2194.619848851441
Optimization restart 3/5, f = -2194.6198506523224
Optimization restart 4/5, f = -2194.6198506532614
Optimization restart 5/5, f = -2194.6198511209345
[[18.         15.7210018  20.26993471 63.59683109  1.          3.33524279]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -2197.5310987225307
Optimization restart 2/5, f = -2197.5310989931754
Optimization restart 3/5, f = -2197.531098855562
Optimization restart 4/5, f = -2197

### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/5, f = -2203.7328964187172
Optimization restart 2/5, f = -2203.7328918431717
Optimization restart 3/5, f = -2203.7328964187604
Optimization restart 4/5, f = -2203.7328963886725
Optimization restart 5/5, f = -2203.7328963973277


GP_regression.,value,constraints,priors
rbf.variance,0.24171293467031288,+ve,
rbf.lengthscale,16.359189204895102,+ve,
Gaussian_noise.variance,0.000429848652739788,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  0.023506310390346074


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  0.027631443315188294


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  0.026195549187910196


In [22]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is 10.266% better than training on random points.'

In [23]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is -5.481% better than training on random points.'

### Save Models

In [24]:
with open(f'experimental_design/models/model_variance_20_init_points_1000_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_1000_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/1020_random_points_supervised_GP_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)

In [26]:
with open(f'experimental_design/models/timeloss-1000-pts.pkl', 'wb') as f:
    pickle.dump((train_X, train_Y), f)